In [1]:
import pandas as pd
import json
import os
import numpy as np
from tqdm import tqdm

In [4]:
try_df = pd.read_csv("immoral(entire).csv")
try_df

,idx,class,conversation,types,intensity,is_immoral
0,0,위협대화,형광색 바지 입는 급식들 왜 이렇게 많아?\nㅈ같은 유행 돌았나보네ㅋㅋ\n형광 바지...,IMMORAL_NONE\nABUSE\nVIOLENCE,"[0, 1.6, 1.8]","[False, True, True]"
1,1,위협대화,자기 몸 긍정 운동가들이라고 하면서 꼴값 떠는 한녀들을 죽여 버리고 싶다.\n자기 ...,HATE\nVIOLENCE\nIMMORAL_NONE\nIMMORAL_NONE\nIM...,"[1.6, 0, 0, 0, 0]","[True, False, False, False, False]"
2,2,위협대화,난 아침에는 항상 시리얼을 먹어\n그딴 과자부스러기를 왜먹어?\n너도 같은 부스러기...,IMMORAL_NONE\nCENSURE\nHATE\nCENSURE,"[0, 1.0, 1.4]","[False, True, True]"
3,3,위협대화,저는 제 와이프가 너무 자랑스러워요\n유일하게 남자들 세계를 잘 이해해주거든요\n자...,IMMORAL_NONE\nIMMORAL_NONE\nCENSURE,"[0, 0, 1.0]","[False, False, True]"
4,4,위협대화,애들은 흙을 좀 만지면서 커야 해.\n그랬다가 나중에 공부라도 못하게 된다면 도시로...,IMMORAL_NONE\nCENSURE\nCENSURE,"[0, 1.4, 1.0]","[False, True, True]"
...,...,...,...,...,...,...
98440,98440,위협대화,울 학교 선배들 벌써부터 군기 잡더라\n머가리 곡괭이로 요단강 내버리고 싶겠네ㅋㅋ\...,IMMORAL_NONE\nVIOLENCE\nVIOLENCE,"[0, 1.8, 1.8]","[False, True, True]"
98441,98441,위협대화,월급 들어오기까지 한참 남았는데 돈 나갈 일이 너무 많네\n너 여자니까 몸이 상품이...,IMMORAL_NONE\nDISCRIMINATION\nHATE\nCENSURE\nS...,"[0, 1.8, 1.8]","[False, True, True]"
98442,98442,위협대화,쟤 아버지가 도박 중독자라며?\n맞아 조만간 빚더미에 앉을걸?\n도박하는 사람들이랑...,CENSURE\nCENSURE\nIMMORAL_NONE\nDISCRIMINATION...,"[1.6, 1.4, 0, 1.6, 1.4]","[True, True, False, True, True]"
98443,98443,위협대화,너 여자친구랑 동거해?\n응응 동거하지\n진짜 징하게 먹겠네,IMMORAL_NONE\nIMMORAL_NONE\nSEXUAL,"[0, 0, 1.8]","[False, False, True]"


### 2.0 이상 있는지 없는지 확인

In [ ]:
def range_intensity(values):
    return any([value >= 2.0 for value in values])

In [ ]:
# df = try_df['intensity'].apply(range_intensity)

### 분리하는 함수 정의하기

In [ ]:
def range_df():
    df = try_df['intensity'].apply(range_intensity)
    df = list(df[df].index) # index만 추출
    df = try_df.iloc[df]
    df = df[['class', 'conversation', 'types']]
    df = df.reset_index().reset_index()
    df.drop(labels="index", axis=1, inplace=True)
    df.rename(columns={'level_0': 'idx'}, inplace=True)
    return df

### 실행

In [ ]:
df_20 = range_df()

### 저장

In [ ]:
df_20.to_csv("more_than_20_immoral.csv", index=False)

### 번외
json2csv

In [ ]:
def make_dataframe(json_data):
    pd.DataFrame(index=range(0, len(json_data)), columns=['idx', 'class','conversation', 'types', 'intensity', 'is_immoral'])

    for i in tqdm(range(len(json_data))):
        text = []
        intensity = []
        is_immoral = []
        types = []
        for j in range(len(json_data[i]['sentences'])):
            text.append(json_data[i]['sentences'][j]['text'])
            intensity.append(json_data[i]['sentences'][j]['intensity'])  # 2.0 <=
            is_immoral.append(json_data[i]['sentences'][j]['is_immoral']) # True
            types += json_data[i]['sentences'][j]['types'] # censure

        # 타입을 평탄화 및 리스트 형태로 변경
        types = np.ndarray.flatten(np.array(types)).tolist()

        # dataframe 에 넣기
        df['idx'][i] = i
        df['class'][i] = "위협대화"
        df['conversation'][i] = "\n".join(text)
        df['types'][i] = "\n".join(types)
        df['intensity'][i] = intensity
        df['is_immoral'][i] = is_immoral
        
    return df

### 위치 지정

In [5]:
json_files = os.listdir("immoral_json_talksets")
json_files.sort()
path = "immoral_json_talksets"

In [6]:
json_files

['talksets-train-1_aihub.json',
 'talksets-train-2.json',
 'talksets-train-3.json',
 'talksets-train-4.json',
 'talksets-train-5.json']

### json 불러오기

In [ ]:
with open(os.path.join(path, json_files[0]), 'r') as f:
    json_data = json.load(f)

### 함수 실행

In [ ]:
dataframe = make_dataframe(json_data)